<a href="https://colab.research.google.com/github/yablockoo/GB/blob/AI/HW4AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:

!pip install --upgrade pip

!pip install tensorflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt


1 - обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете fashion-mnist или cifar10

2 - оценить рост точности при увеличении ширины сети (больше ядер)

3 - оценить рост точности при увеличении глубины сети (больше слоев)

In [57]:

# установка параметров нейросети
batch_size = 28
num_classes = 10
epochs = 5
data_augmentation = False
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [58]:
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

In [59]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

x_train shape: (60000, 28, 28, 1)
60000 тренировочные примеры
10000 тестовые примеры


In [60]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(28, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(28, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout( 0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])


In [61]:
#обучение
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


Epoch 1/5
2143/2143 [==============================] - 15s 7ms/step - loss: 0.9826 - accuracy: 0.6320 - val_loss: 0.6094 - val_accuracy: 0.7653
Epoch 2/5
2143/2143 [==============================] - 10s 5ms/step - loss: 0.6014 - accuracy: 0.7740 - val_loss: 0.5114 - val_accuracy: 0.8080
Epoch 3/5
2143/2143 [==============================] - 10s 5ms/step - loss: 0.5200 - accuracy: 0.8058 - val_loss: 0.4476 - val_accuracy: 0.8386
Epoch 4/5
2143/2143 [==============================] - 10s 5ms/step - loss: 0.4743 - accuracy: 0.8231 - val_loss: 0.4064 - val_accuracy: 0.8539
Epoch 5/5
2143/2143 [==============================] - 11s 5ms/step - loss: 0.4380 - accuracy: 0.8383 - val_loss: 0.3896 - val_accuracy: 0.8581


In [62]:
results = {}
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
results['default'] =  scores[0], scores[1]

313/313 [==============================] - 1s 3ms/step - loss: 0.3896 - accuracy: 0.8581
Test loss: 0.38958117365837097
Test accuracy: 0.8580999970436096


Сделаем сеть поглубже

In [63]:

model1 = Sequential()


model1.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model1.add(Activation('relu'))
model1.add(Conv2D(32, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout( 0.25))

model1.add(Conv2D(64, (3, 3), padding='same'))
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

#добавим еще две свертки и пуллинг
model1.add(Conv2D(64, (3, 3), padding='same'))
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(512))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(num_classes))
model1.add(Activation('softmax'))


model1.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

model1.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


scores = model1.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
results['deeper'] =  scores[0], scores[1]

Epoch 1/5
2143/2143 [==============================] - 12s 5ms/step - loss: 1.6124 - accuracy: 0.3909 - val_loss: 0.8229 - val_accuracy: 0.6844
Epoch 2/5
2143/2143 [==============================] - 12s 6ms/step - loss: 0.8409 - accuracy: 0.6804 - val_loss: 0.6739 - val_accuracy: 0.7417
Epoch 3/5
2143/2143 [==============================] - 11s 5ms/step - loss: 0.6900 - accuracy: 0.7409 - val_loss: 0.5632 - val_accuracy: 0.7871
Epoch 4/5
2143/2143 [==============================] - 11s 5ms/step - loss: 0.6124 - accuracy: 0.7691 - val_loss: 0.5062 - val_accuracy: 0.8103
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.4665 - accuracy: 0.8261
Test loss: 0.4664786756038666
Test accuracy: 0.8260999917984009


In [64]:
model1.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_66 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 activation_92 (Activation)  (None, 28, 28, 32)        0         
                                                                 
 conv2d_67 (Conv2D)          (None, 26, 26, 32)        9248      
                                                                 
 activation_93 (Activation)  (None, 26, 26, 32)        0         
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 dropout_46 (Dropout)        (None, 13, 13, 32)        0         
                                                     

Уберём последний пуллинг дабы избежать уменьшения числа карт признаков.

In [65]:

model1_mod = Sequential()


model1_mod.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model1_mod.add(Activation('relu'))
model1_mod.add(Conv2D(32, (3, 3)))
model1_mod.add(Activation('relu'))
model1_mod.add(MaxPooling2D(pool_size=(2, 2)))
model1_mod.add(Dropout( 0.25))

model1_mod.add(Conv2D(64, (3, 3), padding='same'))
model1_mod.add(Activation('relu'))
model1_mod.add(Conv2D(64, (3, 3)))
model1_mod.add(Activation('relu'))
model1_mod.add(MaxPooling2D(pool_size=(2, 2)))
model1_mod.add(Dropout(0.25))

#добавим еще две свертки
model1_mod.add(Conv2D(64, (3, 3), padding='same'))
model1_mod.add(Activation('relu'))
model1_mod.add(Conv2D(64, (3, 3)))
model1_mod.add(Activation('relu'))
# model1.add(MaxPooling2D(pool_size=(2, 2)))
model1_mod.add(Dropout(0.25))

model1_mod.add(Flatten())
model1_mod.add(Dense(512))
model1_mod.add(Activation('relu'))
model1_mod.add(Dropout(0.5))
model1_mod.add(Dense(num_classes))
model1_mod.add(Activation('softmax'))


model1_mod.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

model1_mod.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


scores = model1_mod.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
results['deeper_mod'] =  scores[0], scores[1]

Epoch 1/5
2143/2143 [==============================] - 13s 6ms/step - loss: 1.2620 - accuracy: 0.5215 - val_loss: 0.6856 - val_accuracy: 0.7336
Epoch 2/5
2143/2143 [==============================] - 12s 6ms/step - loss: 0.6717 - accuracy: 0.7441 - val_loss: 0.5589 - val_accuracy: 0.7871
Epoch 3/5
2143/2143 [==============================] - 12s 6ms/step - loss: 0.5666 - accuracy: 0.7853 - val_loss: 0.4852 - val_accuracy: 0.8147
Epoch 4/5
2143/2143 [==============================] - 12s 6ms/step - loss: 0.5122 - accuracy: 0.8056 - val_loss: 0.4468 - val_accuracy: 0.8302
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.4166 - accuracy: 0.8421
Test loss: 0.41662806272506714
Test accuracy: 0.8421000242233276


In [66]:
model1_mod.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_72 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 activation_100 (Activation)  (None, 28, 28, 32)       0         
                                                                 
 conv2d_73 (Conv2D)          (None, 26, 26, 32)        9248      
                                                                 
 activation_101 (Activation)  (None, 26, 26, 32)       0         
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 dropout_50 (Dropout)        (None, 13, 13, 32)        0         
                                                     

Сделаем сеть пошире

In [67]:

model2 = Sequential()

#увеличим количество нейронов в два раза в каждом слое
model2.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model2.add(Activation('relu'))
model2.add(Conv2D(64, (3, 3)))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout( 0.25))

model2.add(Conv2D(128, (3, 3), padding='same'))
model2.add(Activation('relu'))
model2.add(Conv2D(128, (3, 3)))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(1024))
model2.add(Activation('relu'))
model2.add(Dropout(0.5))
model2.add(Dense(num_classes))
model2.add(Activation('softmax'))


model2.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

model2.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


scores = model2.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
results['wider'] =  scores[0], scores[1]

Epoch 1/5
2143/2143 [==============================] - 15s 7ms/step - loss: 0.9027 - accuracy: 0.6649 - val_loss: 0.5635 - val_accuracy: 0.7834
Epoch 2/5
2143/2143 [==============================] - 14s 7ms/step - loss: 0.5325 - accuracy: 0.8025 - val_loss: 0.4555 - val_accuracy: 0.8343
Epoch 3/5
2143/2143 [==============================] - 14s 7ms/step - loss: 0.4617 - accuracy: 0.8307 - val_loss: 0.4233 - val_accuracy: 0.8450
Epoch 4/5
2143/2143 [==============================] - 14s 7ms/step - loss: 0.4194 - accuracy: 0.8470 - val_loss: 0.3803 - val_accuracy: 0.8592
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.3585 - accuracy: 0.8692
Test loss: 0.3585285246372223
Test accuracy: 0.8691999912261963


In [68]:
result = pd.DataFrame(results, index=['loss', 'accuracy'])
result

,default,deeper,deeper_mod,wider
loss,0.389581,0.466479,0.416628,0.358529
accuracy,0.858100,0.826100,0.842100,0.869200


Опыты показывают, что увеличение количества нейронов в слоях увеличивает точность лучше, чем увеличение количества слоев. Так же увеличение количества слоев может привечти к вырождению количества карт признаков, что ведет к ухудшению точности. Но на больших по размеру картинках следует углублять нейросеть для увеличения точности.  